### Switching probability in first 100 trials
- Unstructured sessions showed higher alpha_chosen and lower alpha_perseverance compared to structured sessions, suggesting animals are more likely to switch in the unstructured sessions.

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from banditpy.analyses import SwitchProb2Arm

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100)
    switch_prob = SwitchProb2Arm(task).by_session()
    print(f"Switch prob: {switch_prob}")

    df = pd.DataFrame(
        {
            "switch_prob": [switch_prob],
            "name": exp.sub_name,
            "grp": "struc" if task.is_structured else "unstruc",
            "first_experience": True if "Exp1" in exp.sub_name else False,
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_100trials")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
from mab_colors import colors_2arm
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 4, size=(11, 3), num=1, fontsize=10)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_100trials
df = df[df["first_experience"] == True]

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="grp", y="switch_prob", hue="grp", ax=ax)
sns.stripplot(
    palette=["gray", "gray"],
    edgecolor="white",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    # errorbar="se",
    **plot_kw,
)
sns.barplot(
    palette=colors_2arm(),
    # edgecolor="0.5",
    # facecolor=(0, 0, 0, 0),
    alpha=0.6,
    errorbar="se",
    **plot_kw,
)


orders = ["unstruc", "struc"]
pairs = [(("unstruc"), ("struc"))]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()

ax.set_title("Switch probability")
# ax.legend("")

### Switching probability as a function of trials within session
- Check if switch probability curves for unstructured and structured sessions have same decay and tail

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d
from banditpy.analyses import SwitchProb2Arm


exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100).filter_by_deltaprob(
        delta_min=0.05
    )
    switch_prob = SwitchProb2Arm(task).by_trial()
    switch_prob_smooth = gaussian_filter1d(switch_prob, sigma=1)

    df = pd.DataFrame(
        {
            "trial_id": np.arange(len(switch_prob)) + 1,
            "switch_prob": switch_prob,
            "switch_prob_smooth": switch_prob_smooth,
            "name": exp.sub_name,
            "grp": "struc" if task.is_structured else "unstruc",
            "first_experience": True if "Exp1" in exp.sub_name else False,
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_by_trial_100trials")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw
from mab_colors import colors_2arm

fig = plotting.Fig(4, 2, size=(8.5, 11), num=1, fontsize=10)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_by_trial_100trials
df = df[df["first_experience"] == True]

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob", hue="grp", ax=ax)
sns.lineplot(
    palette=colors_2arm(),
    # palette=["#E89317", "#3980ea"],
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)

orders = np.arange(1, 100)
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders[::10]]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()

ax_ = fig.subplot(fig.gs[1])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob_smooth", hue="grp", ax=ax_)
sns.lineplot(
    palette=colors_2arm(),
    # palette=["#E89317", "#3980ea"],
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)


ax.set_title("Switch probability")
ax.get_legend().remove()
ax.set_ylim(0.02, 0.17)
ax.set_xlim(1, 100)
ax.set_xticks([1, 25, 50, 75, 100])

### As a function of trials within session (First exposure)
- Check if switch probability curves for unstructured and structured sessions have same decay and tail

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d


exps = mab_subjects.unstruc.first_exposure + mab_subjects.struc.first_exposure

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.filter_by_trials(min_trials=100, clip_max=100)
    switch_prob = mab.get_switch_prob_by_trial()
    switch_prob_smooth = gaussian_filter1d(switch_prob, sigma=1)

    df = pd.DataFrame(
        {
            "trial_id": np.arange(len(switch_prob)) + 1,
            "switch_prob": switch_prob,
            "switch_prob_smooth": switch_prob_smooth,
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_by_trial_100trials_first_exposure")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(8, 4, size=(8.5, 11), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_by_trial_100trials_first_exposure

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob", hue="grp", ax=ax)
sns.lineplot(
    palette="husl",
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)

orders = np.arange(1, 100)
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders[::10]]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()

ax_ = fig.subplot(fig.gs[1])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob_smooth", hue="grp", ax=ax_)
sns.lineplot(
    palette="husl",
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)


ax.set_title("Switch probability \n (First experience)")
ax.get_legend().remove()
ax.set_ylim(0.02, 0.17)
ax.set_xlim(1, 100)
ax.set_xticks([1, 25, 50, 75, 100])

### Beron et al. 2022
- It will be interesting to assess switch probability based on history and compare between unstruc and struc.
- Do it for easy and hard sessions as well

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from banditpy.analyses import SwitchProb2Arm

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100).filter_by_deltaprob(
        delta_min=0.05
    )
    sp, seq = SwitchProb2Arm(task).by_history(3, history_as_str=True)

    task_easy = task.filter_by_deltaprob(delta_min=0.4)
    sp_easy, seq_easy = SwitchProb2Arm(task_easy).by_history(3, history_as_str=True)

    task_hard = task.filter_by_deltaprob(delta_min=0, delta_max=0.3)
    sp_hard, seq_hard = SwitchProb2Arm(task_hard).by_history(3, history_as_str=True)

    assert np.all(seq == seq_easy), "sequences are not the same"
    assert np.all(seq_hard == seq_easy), "Easy and hard sequences are not the same"

    df = pd.DataFrame(
        {
            "switch_prob": sp,
            "switch_prob_easy": sp_easy,
            "switch_prob_hard": sp_hard,
            "seq": seq,
            "name": exp.sub_name,
            "grp": "struc" if task.is_structured else "unstruc",
            "first_experience": True if "Exp1" in exp.sub_name else False,
        }
    )
    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_seq")

In [ ]:
from neuropy import plotting
import mab_subjects
from statplotannot.plots import SeabornPlotter
from mab_colors import colors_2arm
import numpy as np


df = mab_subjects.GroupData().switch_prob_seq
df = df[df["first_experience"] == True]


fig = plotting.Fig(8, 3, size=(8.5, 11), num=1)
titles = ["in all sessions", "in easy sessions", "in hard sessions"]
for i, level in enumerate(["switch_prob", "switch_prob_easy", "switch_prob_hard"]):

    ax = fig.subplot(fig.gs[i, :2])
    SeabornPlotter(
        data=df,
        x="seq",
        y=level,
        hue="grp",
        hue_order=["unstruc", "struc"],
        ax=ax,
    ).barplot(
        dodge=False, palette=colors_2arm(1), alpha=0.6, errorbar="se"
    ).bootstrap_test(
        statistic=np.mean, verbose=False
    )

    ax.set_xlabel("")
    ax.set_title(f"Switch probability {titles[i]} \n (FE)")
    ax.tick_params("x", rotation=90)
    ax.set_ylabel("P(Switch)")
    ax.set_ylim(0, 0.75)

### SP after consecutively rewarded and unrewarded trials. 
- Assess switch probability when animals are rewarded/unrewarded for 1, 2, or 3 consecutive trials.

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from banditpy.analyses import SwitchProbability2AB

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.filter_by_trials(min_trials=100, clip_max=100)
    sp = SwitchProbability2AB(mab)
    sp1, seq1 = sp.by_history(1, history_as_str=True)
    sp2, seq2 = sp.by_history(2, history_as_str=True)
    sp3, seq3 = sp.by_history(3, history_as_str=True)

    req_seq = np.array(["A", "AA", "AAA", "a", "aa", "aaa"])
    req_prob = [
        sp1[seq1 == "A"][0],
        sp2[seq2 == "AA"][0],
        sp3[seq3 == "AAA"][0],
        sp1[seq1 == "a"][0],
        sp2[seq2 == "aa"][0],
        sp3[seq3 == "aaa"][0],
    ]

    df = pd.DataFrame(
        {
            "switch_prob": req_prob,
            "seq": req_seq,
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
            "first_exposure": True if "Exp1" in exp.sub_name else False,
        }
    )
    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_consecutive")

In [ ]:
from neuropy import plotting
import mab_subjects
import numpy as np
from statplotannot.plots import SeabornPlotter
from statplotannot.plots.colormaps import colors_mab


df = mab_subjects.GroupData().switch_prob_consecutive
df = df[df["first_exposure"] == True]

fig = plotting.Fig(8, 3, num=1)
ax = fig.subplot(fig.gs[0])

SeabornPlotter(
    data=df, x="seq", y="switch_prob", hue="grp", hue_order=["unstruc", "struc"], ax=ax
).barplot(dodge=True, palette=colors_mab(1), alpha=0.8, errorbar="se").bootstrap_test(
    statistic=np.mean
)

ax.set_title(
    "Switch probability following\n consecutively rewarded or unrewarded trial \n (First Experience)"
)
ax.tick_params("x", rotation=90)
ax.set_ylabel("P(Switch)")
ax.set_xlabel("History sequence")
# ax.set_ylim(0, 0.75)
ax.get_legend().remove()

In [ ]:
import numpy as np
from scipy.optimize import minimize


class UCB2ArmedBandit:
    def __init__(self, n_trials, alpha=0.1, beta=1.0, c=1.0):
        self.n_trials = n_trials
        self.alpha = alpha  # learning rate
        self.beta = beta  # inverse temperature
        self.c = c  # UCB exploration parameter

    def simulate(self, rewards):
        """
        Simulate choices using UCB algorithm.
        rewards: shape (n_trials, 2) - reward for each arm at each trial
        Returns: choices (0 or 1), Q-values, UCB values
        """
        Q = np.zeros(2)
        N = np.zeros(2)
        choices = []
        Q_hist = []
        UCB_hist = []

        for t in range(self.n_trials):
            ucb = Q + self.c * np.sqrt(np.log(t + 1) / (N + 1e-5))
            p = np.exp(self.beta * ucb) / np.sum(np.exp(self.beta * ucb))
            choice = np.random.choice([0, 1], p=p)
            reward = rewards[t, choice]
            N[choice] += 1
            Q[choice] += self.alpha * (reward - Q[choice])
            choices.append(choice)
            Q_hist.append(Q.copy())
            UCB_hist.append(ucb.copy())
        return np.array(choices), np.array(Q_hist), np.array(UCB_hist)

    def neg_log_likelihood(self, params, choices, rewards):
        alpha, beta, c = params
        Q = np.zeros(2)
        N = np.zeros(2)
        nll = 0
        for t in range(len(choices)):
            ucb = Q + c * np.sqrt(np.log(t + 1) / (N + 1e-5))
            p = np.exp(beta * ucb) / np.sum(np.exp(beta * ucb))
            nll -= np.log(p[choices[t]] + 1e-10)
            reward = rewards[t, choices[t]]
            N[choices[t]] += 1
            Q[choices[t]] += alpha * (reward - Q[choices[t]])
        return nll

    def fit(self, choices, rewards, bounds=[(0.01, 1), (0.1, 10), (0.01, 5)]):
        """
        Fit UCB parameters to observed choices and rewards.
        Returns: dict of fitted parameters
        """

        def obj(params):
            return self.neg_log_likelihood(params, choices, rewards)

        x0 = [0.2, 1.0, 1.0]
        res = minimize(obj, x0, bounds=bounds, method="L-BFGS-B")
        return {"alpha": res.x[0], "beta": res.x[1], "c": res.x[2], "nll": res.fun}


# Example usage:
rewards = np.random.binomial(1, [0.7, 0.3], size=(100, 2))
model = UCB2ArmedBandit(n_trials=100)
choices, Q_hist, UCB_hist = model.simulate(rewards)
fit_result = model.fit(choices, rewards)
print(fit_result)

### SP as a function of trials and distinguishing by blocks

In [1]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d
from banditpy.analyses import SwitchProb2Arm
from datetime import datetime as dt


exps = mab_subjects.unstruc.allsess
# exps = mab_subjects.unstruc.BratExp1

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=60, clip_max=60)
    # switch_prob = SwitchProb2Arm(task).by_trial()
    # switch_prob_smooth = gaussian_filter1d(switch_prob, sigma=1)
    is_start = task.is_session_start
    start_datetime = task.datetime[is_start]
    start_times = [dt.strptime(_, "%Y-%m-%d %H:%M:%S") for _ in start_datetime]
    timedelta = (
        np.diff(start_times, prepend=start_times[0])
        .astype("timedelta64[s]")
        .astype(int)
    )
    is_block = np.where(timedelta < 2400, 1, 0)

    block_id = []
    block_id_flag = 0
    for val in is_block:
        if val:
            block_id_flag += 1
        else:
            block_id_flag = 1
        block_id.append(block_id_flag)
    block_id = np.array(block_id)

    print(np.unique(block_id, return_counts=True))
    for idx in [1, 2, 3]:
        if np.sum(block_id == idx) >= 3:
            block_task = task.filter_by_session_id(task.sessions[block_id == idx])
            swp = SwitchProb2Arm(block_task).by_trial()[:60]

            df = pd.DataFrame(
                {
                    "trial_id": np.arange(len(swp)) + 1,
                    "switch_prob": swp,
                    "block_id": idx,
                    "name": exp.sub_name,
                    "grp": "struc" if task.is_structured else "unstruc",
                    "first_experience": True if "Exp1" in exp.sub_name else False,
                }
            )

            swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_blockwise")

AggroExp1Unstructured
(array([1, 2, 3, 4, 5, 6, 7]), array([204, 188, 150,  87,  40,  17,   5]))
AuromaExp1Unstructured
(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([94, 85, 68, 48, 31, 16, 15,  9,  4,  2]))
BratExp1Unstructured
(array([1, 2, 3]), array([126,  34,   3]))
GronckleExp2Unstructured
(array([1, 2, 3, 4, 5, 6]), array([273, 230, 153,  68,  16,   1]))
GrumpExp1Unstructured
(array([1, 2, 3, 4]), array([134,   1,   1,   1]))
ToothlessExp2Unstructured
(array([1, 2, 3, 4, 5, 6]), array([247, 213, 169, 115,  57,  18]))
switch_prob_blockwise saved


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw
from mab_colors import colors_2arm

fig = plotting.Fig(5, 2, size=(8.5, 11), num=1, fontsize=10)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_blockwise
df = df[df["first_experience"] == True]

ax = fig.subplot(fig.gs[0])

plot_kw = dict(data=df, x="trial_id", y="switch_prob", hue="block_id", ax=ax)
sns.lineplot(
    # estimator=np.median,
    errorbar="se",
    # ci=60,
    err_kws={"edgecolor": None},
    # palette=["r", "g", "b"],
    palette="Dark2",
    **plot_kw,
)
ax.set_title("Switch probability (blockwise)")
# ax.get_legend().remove()
# ax.set_ylim(0.02, 0.17)
ax.set_xlim(1, 60)
ax.set_xticks([1, 20, 40, 60])

# sns.stripplot(
#     palette=["r", "g", "b"],
#     **plot_kw,
# )

names = df["name"].unique()

for i, name in enumerate(names):
    df_name = df[df["name"] == name]

    ax = fig.subplot(fig.gs[i, 1])
    sns.lineplot(
        data=df_name,
        x="trial_id",
        y="switch_prob",
        hue="block_id",
        errorbar="se",
        # ci=60,
        err_kws={"edgecolor": None},
        # palette=["r", "g", "b"],
        palette="Dark2",
    )
    ax.set_title(f"Subject:{name}")